In [76]:
class Tree:
    """Abstract base class representing a tree structure."""
    
    #------------------------------- nested Position class -------------------------------
    class Position:
        """An abstraction representing the location of a single element."""
        def element(self):
            """Return the element stored at this Position."""
            raise NotImplementedError( 'must be implemented by subclass' )

        def __eq__ (self, other):
            """Return True if other Position represents the same location."""
            raise NotImplementedError( 'must be implemented by subclass' )
       
        def __ne__ (self, other):
            """Return True if other does not represent the same location."""
            return not (self == other)              # opposite of     eq
       
    
    # ---------- abstract methods that concrete subclass must support ----------
    
    def root(self):
        """Return Position representing the tree s root (or None if empty)."""
        raise NotImplementedError( 'must be implemented by subclass' )
    
    def parent(self, p):
        """Return Position representing p s parent (or None if p is root)."""
        raise NotImplementedError( 'must be implemented by subclass' )
    
    def num_children(self, p):
        """Return the number of children that Position p has."""
        raise NotImplementedError( 'must be implemented by subclass' )
    
    def children(self, p):
        """Generate an iteration of Positions representing p s children."""
        raise NotImplementedError( 'must be implemented by subclass' )
    
    def __len__ (self):
        """Return the total number of elements in the tree."""
        raise NotImplementedError( 'must be implemented by subclass' )
        
    def   _height1(self):                                         # works, but O(nˆ2) worst-case time
        """Return the height of the tree."""
        return max(self.depth(p) for p in self.positions( ) if self.is_leaf(p)) 
    
    def   height2(self, p): # time is linear in size of subtree
        """Return the height of the subtree rooted at Position p."""
        if self.is_leaf(p):
            return 0
        else:
            return 1 + max(self._height2(c) for c in self.children(p))
    
    def positions(self):
        """Generate an iteration of the tree s positions."""
        return self.preorder( )
    
    def __iter__ (self):
        """Generate an iteration of the tree s elements."""
        for p in self.positions(): # use same order as positions()
            yield p.element( )    # but yield each element
        
    # ---------- concrete methods implemented in this class ----------
    
    def is_root(self, p):
        """Return True if Position p represents the root of the tree."""
        return self.root( ) == p
    
    def is_leaf(self, p):
        """Return True if Position p does not have any children."""
        return self.num_children(p) == 0
    
    def is_empty(self):
        """Return True if the tree is empty."""
        return len(self) == 0  
    
    def depth(self, p):
        """Return the number of levels separating Position p from the root."""
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))
    
    def height(self, p=None):
        """Return the height of the subtree rooted at Position p.
        If p is None, return the height of the entire tree. """
        if p is None:
            p = self.root()
        return self. height2(p) # start height2 recursion
    
    def preorder(self):
        """Generate a preorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):   # start recursion
                yield p

    def _subtree_preorder(self,p):
        """Generate a preorder iteration of positions in subtree rooted at p."""
        yield p
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other            # yielding each to our caller


    def postorder(self):
        """Generate a postorder iteration of positions in the tree.""" 
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):   # start recursion
                yield p

    def _subtree_postorder(self, p):
        """Generate a postorder iteration of positions in subtree rooted at p."""
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other 
        yield p 
        
    def breadthfirst(self):
        """Generate a breadth-first iteration of the positions of the tree."""
        if not self.is_empty():
            fringe = LinkedQueue()
            fringe.enqueue(self.root( ))
        while not fringe.is_empty( ):
            p = fringe.dequeue( ) 
            yield p
            for c in self.children(p):
                fringe.enqueue(c)  
                
    def __str__(self):
        for p in self.preorder():
            print(p.element())

In [77]:
class BinaryTree(Tree):
    """Abstract base class representing a binary tree structure."""

    # --------------------- additional abstract methods ---------------------
    def left(self, p):
        """Return a Position representing p s left child.
         Return None if p does not have a left child.
          """
        raise NotImplementedError( 'must be implemented by subclass' )

    def right(self, p):
        """Return a Position representing p s right child.
        Return None if p does not have a right child.
        """
        raise NotImplementedError( 'must be implemented by subclass' )
        
        
    # ---------- concrete methods implemented in this class ----------
    def sibling(self, p):
        """Return a Position representing p s sibling (or None if no sibling)."""
        parent = self.parent(p)
        if parent is None:           # p must be the root
            return None               # root has no sibling
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
            
    # override inherited version to make inorder the default
    def positions(self):
        """Generate an iteration of the tree s positions."""
        return self.inorder( )        

    def children(self, p):
        """Generate an iteration of Positions representing p s children."""
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)  
            
    def inorder(self):
        """Generate an inorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p
    
    def _subtree_inorder(self, p):
        """Generate an inorder iteration of positions in subtree rooted at p."""
        if self.left(p) is not None: # if left child exists, traverse its subtree
            for other in self._subtree_inorder(self.left(p)):
                yield other
                yield p # visit p between its subtrees
                if self.right(p) is not None: # if right child exists, traverse its subtree
                    for other in self._subtree_inorder(self.right(p)):
                        yield other        

In [94]:
class LinkedBinaryTree(BinaryTree):
    """Linked representation of a binary tree structure."""
    
    class _Node:# Lightweight, nonpublic class for storing a node.
        __slot__ =  '_element' ,' _parent' , '_left' , '_right'
        def __init__(self, element, parent=None, left=None,right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right
    
    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element"""
        
        def __init__(self,container,node):
            """Constructor should not be invoked by user."""
            self._container = container
            self._node = node

        def element(self):
            """Return the element stored at this Position."""
            return self._node._element
        
        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node
        
    def _validate(self,p):
        """Return associated node, if position is valid."""
        if not instance(p, self.Position):
            return TypeError('p must be proper Position type')
        if p._container is not self:
            return ValueError('p does not belong to this container')
        if p._node._parent is p._node:   # convention for deprecated nodes
            return ValueError('p is no longer valid')
        return p._node

    def _make_position(self,node):
        """Return Position instance for given node (or None if no node)."""
        return self.Position(self,node) if node is not None else None
        
        
    #-------------------------- binary tree constructor --------------------------
    def __init__(self):
        """Create an initially empty binary tree."""
        self._root = None
        self._size = 0

    #-------------------------- public accessors --------------------------
    def __len__(self):
        """Return the total number of elements in the tree."""
        return self._size

    def root(self):
        """Return the root Position of the tree (or None if tree is empty)."""
        return self._make_position(self._root)

    def parent(self,p):
        """Return the Position of p's parent (or None if p is root)."""
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self,p):
        """Return the Position of p's left child (or None if no left child)."""
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self,p):
        """Return the Position of p's right child (or None if no right child)."""
        node = self._validate(p)
        return self._make_position(node._right)
        
    def num_children(self,p):
        """Return the number of children of Position p."""
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count

    def _add_root(self,e):
        """
        Place element e at the root of an empty tree and return new Position.
        Raise ValueError if tree nonempty.
        """
        if self._root is not None: raise ValueError('Root exist')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self,p,e):
        """
        Create a new left child for Position p, storing element e.
        Return the Position of new node.
        Raise ValueError if Position p is invalid or p already has a left child.
        """
        node = self._validate(p)
        if self._left is not None: raise ValueError('left child exist')
        self._size += 1
        self._left = self._Node(e)         # node is its parent
        return self._make_position(node._left)

    def _add_right(self,p,e):
        """
        Create a new right child for Position p, storing element e.
        Return the Position of new node.
        Raise ValueError if Position p is invalid or p already has a right child.
        """
        node = self._validate(p)
        if self._right is not None: raise ValueError('right child exist')
        self._size += 1
        self._right = self._Node(e)       # node is its parent
        return self._make_position(node._right)


    def _replace(self,p,e):
        """Replace the element at position p with e, and return old element."""
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

    def _delete(self,p):
        """Delete the node at Position p, and replace it with its child, if any.
        Return the element that had been stored at Position p.
        Raise ValueError if Position p is invalid or p has two children."""

        node = self._validate(p)
        if self.num_children(p)==2: raise ValueError('p has two children')
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent    # child s grandparent becomes parent

        if node is self._root:
            self._root = child

        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node      # convention for deprecated node
        return node._element
        
    def   _attach(self, p, t1, t2):
        """Attach trees t1 and t2 as left and right subtrees of external p."""
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError( 'position must be leaf' )
        if not type(self) is type(t1) is type(t2): # all 3 trees must be same type
            raise TypeError( 'Tree types must match' )
        self._size += len(t1) + len(t2)
        if not t1.is_empty():        # attached t1 as left subtree of node
            t1._root._parent = node
            node._left = t1._root
            t1._root = None   # set t1 instance to empty
            t1_size=0
        if not t2.is_empty():     # attached t2 as left subtree of node
            t2._root._parent = node
            node._right = t2._root
            t2._root = None      # set t2 instance to empty
            t2_size=0

                
        
        
                

In [97]:
LBT = LinkedBinaryTree()
LBT._add_root(76)
#LBT._add_left(LBT.node,7)


# for i in iter(LBT):
#     print(i.element())



In [50]:
class Tree2():
    """Linked representation of a binary tree structure."""
       
    __slot__ =  '_element' ,' _parent' , '_left' , '_right'

    def __init__(self, element, parent=None, left=None,right=None):
        self._element = element
        self._parent = parent
        self._left = left
        self._right = right
        self._size = 0
       
    def insert(self,e):
        """Add element e to the Tree."""
        if self._element:
            if e < self._element:
                if self._left is None:
                    self._left = Tree2(e)
                else:
                    self._left.insert(e)
            elif e > self._element:
                if self._right is None:
                    self._right = Tree(e)
                else:
                    self._right.insert(e)                    
        else:
            self._element
        self._size += 1
        
    def __len__(self):
        return self._size

    def is_empty(self):
        return len(self) == 0

    def __str__(self):
        if self._left:
            self._left.__str__()
        return str(self._element)
        if self._right:
            self._right.__str__()
            
T= Tree2(12)
#T._Node(12,2,1,2)
len(T)
for i in range(5): T.insert(i)
len(T)
print(T)


12


In [234]:
class Node:
    def __init__(self, element):
        self._left = None
        self._right = None
        self._element = element
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
 
    def insert(self, e):
        """Compare the new value with the parent node"""
        if self._element:
            if e < self._element:
                if self._left is None:
                    self._left = Node(e)
                else:
                    self._left.insert(e)
            elif e > self._element:
                if self._right is None:
                    self._right = Node(e)
                else:
                    self._right.insert(e)
        else:
            self._element
        self._size += 1
           
        
    # Print the tree
    def PrintTree(self):
        if self._left is not None:
            self._left.PrintTree()
        print (self._element),
        if self._right is not None:
            self._right.PrintTree()         
            
            
root = Node(12)
root.insert(6)
root.insert(14)
root.insert(3)
print('length =',root._size)

root.PrintTree()
len(root)

length = 3
3
6
12
14


3

In [191]:
# Inorder traversal
# Left -> Root -> Right

class inorder(Node):
    def inorder(self,root):
        result = []
        if root is not None:
            result = self.inorder(root._left)
            result.append(root._element)
            result = result + self.inorder(root._right)
        return result

root = inorder(12)
root.insert(6)
root.insert(14)
root.insert(3)
root.PrintTree()
print(root.inorder(root))

root = inorder(27)
root.insert(14)
root.insert(35)
root.insert(10)
root.insert(19)
root.insert(31)
root.insert(42)
root.PrintTree()
print(root.inorder(root))

3
6
12
14
[3, 6, 12, 14]
10
14
19
27
31
35
42
[10, 14, 19, 27, 31, 35, 42]


In [192]:
# Pre-order traversal
#  Root ->Left -> Right

class pre_order(Node):
    def pre_order(self,root):
        result = []
        if root is not None:
            result.append(root._element)
            result = result + self.pre_order(root._left)
            result = result + self.pre_order(root._right)
        return result

root = pre_order(12)
root.insert(6)
root.insert(14)
root.insert(3)
root.PrintTree()
print(root.pre_order(root))

root = pre_order(27)
root.insert(14)
root.insert(35)
root.insert(10)
root.insert(19)
root.insert(31)
root.insert(42)
root.PrintTree()
print(root.pre_order(root))

3
6
12
14
[12, 6, 3, 14]
10
14
19
27
31
35
42
[27, 14, 10, 19, 35, 31, 42]


In [193]:
# Post-order traversal
#  Left -> Right-> Root 

class post_order(Node):
    def post_order(self,root):
        result = []
        if root is not None:
            result = self.post_order(root._left)
            result = result + self.post_order(root._right)
            result.append(root._element)            
        return result
    

root = post_order(12)
root.insert(6)
root.insert(14)
root.insert(3)
root.PrintTree()
print(root.post_order(root))

root = post_order(27)
root.insert(14)
root.insert(35)
root.insert(10)
root.insert(19)
root.insert(31)
root.insert(42)
root.PrintTree()
print(root.post_order(root))

3
6
12
14
[3, 6, 14, 12]
10
14
19
27
31
35
42
[10, 19, 14, 31, 42, 35, 27]


In [194]:
#-------------R-8-1----------------
"""a) root - node which has no parent
   b) internal nodes - nodes with one or more children
   c) descendants does node cs016/ have - 6 hw1,hw2,hw3,pr1,pr2,pr3
   d) ancestors does node cs016/ have  - none
   e) siblings of node homeworks/ - programs/
   f) Which nodes are in the subtree rooted at node projects/? - papers/ , demos/
   g) What is the depth of node papers/? root - 0, else-depth of p is one plus the depth of the parent - 3
   h) What is the height of the tree? p is leaf - 0 , 1+ max height of p's children -  4
   
"""

"a) root - node which has no parent\n   b) internal nodes - nodes with one or more children\n   c) descendants does node cs016/ have - 6 hw1,hw2,hw3,pr1,pr2,pr3\n   d) ancestors does node cs016/ have  - none\n   e) siblings of node homeworks/ - programs/\n   f) Which nodes are in the subtree rooted at node projects/? - papers/ , demos/\n   g) What is the depth of node papers/? root - 0, else-depth of p is one plus the depth of the parent - 3\n   h) What is the height of the tree? p is leaf - 0 , 1+ max height of p's children -  4\n   \n"

In [195]:
#-------------R-8-2----------------
class maxDepth(Node):
    #maxdepth = height 
    def height(self,root):
            if root is None:
                return 0
            else:
                left_Depth = self.height(root._left)
                right_Depth = self.height(root._right)
                
            return max(left_Depth,right_Depth) + 1
        
    def depth(self,root):
        if root is None:
            return 0
        else:
            return 1+ self.depth(root)

root = maxDepth(27)
root.insert(14)
root.insert(35)
root.insert(10)
root.insert(19)
root.insert(31)
#root.insert(42)
node = root.insert(42)
print('depth =', root.depth(node))
root.PrintTree()
print('height =',root.height(root))   

"""O(nˆ2) is worst case """

depth = 0
10
14
19
27
31
35
42
height = 3


'O(nˆ2) is worst case '

In [196]:
#-------------R-8-3----------------
"""maxdepth == height of the Tree"""

'maxdepth == height of the Tree'

In [197]:
#-------------R-8-4----------------
"""
def   height2(self, p): # time is linear in size of subtree
    if self.is leaf(p):
    return 0
    else:
    return 1 + max(self. height2(c) for c in self.children(p))
    
    O(n)
"""

'\ndef   height2(self, p): # time is linear in size of subtree\n    if self.is leaf(p):\n    return 0\n    else:\n    return 1 + max(self. height2(c) for c in self.children(p))\n    \n    O(n)\n'

In [198]:
#-------------R-8-5----------------
class countLeft(Node):
    def countLeft(self,root):
        result = []
        if root is not None:
            result = self.countLeft(root._left)
            result.append(root._element)
        return result    

        
root = countLeft(27)
root.insert(14)
root.insert(35)
root.insert(10)
root.insert(19)
root.insert(31)
root.insert(42) 
root.PrintTree()
root.countLeft(root)

10
14
19
27
31
35
42


[10, 14, 27]

In [199]:
#-------------R-8-6----------------
"""
proper -if each node has either zero or two children
Let T be an n-node binary tree that may be improper. 

For proper binary tree n+1 node 
"""

'\nLet T be an n-node binary tree that may be improper. \n\nFor proper binary tree n+1 node \n'

In [ ]:
#-------------R-8-7----------------
"""
for n internal nodes n+1 external nodes for improper binary tree with n nodes

induction of n , x(n) number of external nodes 
base case X(0) -- 1 -- n+1
theorum is true for all i<n
because n>=1
X(n) = X(k)+X(n-k-1)
     = k+1 + n-k-1+1
     = n+1
"""


In [201]:
#-------------R-8-8----------------
"""
a. What is the minimum number of external nodes for a proper binary tree with height h? Justify your answer.
h+1≤nE ≤2h

mininum number of nE would be h+1 
justification - let proper binary tree has node = 4 has height log(n) + 1 to n which is 2 to 4,

              0
            /  \
           0    0
          / \   
         0   0    . nE number of nodes with no children , height of this tree is 2 hence nE = h+1 = 3 

b. What is the maximum number of external nodes for a proper binary tree with height h? Justify your answer.

Justification
max number of nE would be 2h , h = 3 2*3 = 6 nE = 4 <=6

              0
            /  \
           0    0
          / \   /\
         0   0 0  0
        

c. Let T be a proper binary tree with height h and n nodes. Show that log(n+1)−1 ≤ h ≤ (n−1)/2.

node of i is 2^i with h height (2^h+1) -1 , n <= (2^h+1) -1
n + 1 <= (2^h+1)
log(n+1) <= h+1
h >= log(n+1) -1

1 + 2^1 + 2^2 + 2^3 + ... + 2^h−1 = n
n = 2^h-1
2^h = n + 1
log(2^h) = log(n+1)
h = log(n+1)  --- h is O(logn)

number of leaves in tree = lh = 2^(h-1)
2^h/2^1 --> 2^log(n+1)-1/2 ---> n-1/2
h/2

d. For which values of n and h can the above lower and upper bounds on h be attained with equality?
lower = 1
upper = n+1

"""


'\na. What is the minimum number of external nodes for a proper binary tree with height h? Justify your answer.\nh+1≤nE ≤2h\n\nmininum number of nE would be h+1 \njustification - let proper binary tree has node = 4 has height log(n) + 1 to n which is 2 to 4,\n\n              0\n            /             0    0\n          / \\   \n         0   0    . nE number of nodes with no children , height of this tree is 2 hence nE = h+1 = 3 \n\nb. What is the maximum number of external nodes for a proper binary tree with height h? Justify your answer.\n\nJustification\nmax number of nE would be 2h , h = 3 2*3 = 6 nE = 4 <=6\n\n              0\n            /             0    0\n          / \\   /         0   0 0  0\n        \n\nc. Let T be a proper binary tree with height h and n nodes. Show that log(n+1)−1 ≤ h ≤ (n−1)/2.\n\nnode of i is 2^i with h height (2^h+1) -1 , n <= (2^h+1) -1\nn + 1 <= (2^h+1)\nlog(n+1) <= h+1\nh >= log(n+1) -1\n\n1 + 2^1 + 2^2 + 2^3 + ... + 2^h−1 = n\nn = 2^h-1\n2^h = n +

In [202]:
#-------------R-8-9----------------
"""
induction of n , x(n) number of external nodes 
base case X(0) -- 1 -- n+1
theorum is true for all i<n
because n>=1
X(n) = X(k)+X(n-k-1)
     = k+1 + n-k-1+1
     = n+1
"""


'\ninduction of n , x(n) number of external nodes \nbase case X(0) -- 1 -- n+1\ntheorum is true for all i<n\nbecause n>=1\nX(n) = X(k)+X(n-k-1)\n     = k+1 + n-k-1+1\n     = n+1\n'

In [224]:
#-------------R-8-9----------------
class _Node:
    def __init__(self, element):
        self._left = None
        self._right = None
        self._element = element

class BTree:
    def __init__(self):
        self._root = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0

    def getRoot(self):
        return self._root

    def add(self, element):
        if self._root is None:
            self._root = _Node(element)
        else:
            self._add(element, self._root)
        self._size += 1    

    def _add(self, element, node):
        if element < node._element:
            if node._left is not None:
                self._add(element, node._left)
            else:
                node._left = _Node(element)
        else:
            if node._right is not None:
                self._add(element, node._right)
            else:
                node._right = _Node(element)
                

    def find(self, element):
        if self._root is not None:
            return self._find(element, self._root)
        else:
            return None

    def _find(self, element, node):
        if element == node._element:
            return node
        elif (element < node._element and node.l is not None):
            return self._find(element, node._left)
        elif (element > node._element and node._right is not None):
            return self._find(element, node._right)

    def deleteTree(self):
        # garbage collector will do this for us. 
        self._root = None

    def printTree(self):
        if self._root is not None:
            self._printTree(self._root)

    def _printTree(self, node):
        if node is not None:
            self._printTree(node._left)
            print(str(node._element) + ' ')
            self._printTree(node._right)
  

        
node = BTree()
node.add(27)
node.add(14)
node.add(35)
node.add(10)
node.add(19)
node.add(31)
node.add(42) 
node.printTree()

node.is_empty()
len(node)


10 
14 
19 
27 
31 
35 
42 


7

In [219]:
#-------------R-8-10----------------
class childrenCount(BTree):
    def num_children(self,node):
        count = 0
        if node._root._left is not None:
            count += 1
        if node._root._right is not None:    
            count += 1
        return count  
    
node = childrenCount()
node.add(27)
node.add(14)
node.add(35)
node.add(10)
node.add(19)
node.add(31)
node.add(42) 
node.printTree() 
print(node.num_children(node))

10 
14 
19 
27 
31 
35 
42 
2


In [225]:
#-------------R-8-11----------------
"""
subtree 1 from left = (4*3) = 12
subtree 2 = (4+2)=6
subtree 3 = (3*3) = 9

"""

'\nsubtree 1 from left = (4*3) = 12\nsubtree 2 = (4+2)=6\nsubtree 3 = (3*3) = 9\n\n'

In [226]:
#-------------R-8-12----------------
"""
            (21)
          /.    \
        (-)      (-) return
       /   \       \
      (*)  (7)     (1)
    /   \
   (5)   (6)
   
"""   

'\n            (21)\n          /.            (-)      (-) return\n       /   \\             (*)  (7)     (1)\n    /      (5)   (6)\n   \n'

In [ ]:
#-------------R-8-13----------------
"""
(((5+2)*(2-1))/((2+9)+((7-2)-1))*8)

                                   (3.7)
                            /               \
                          (/)               (*)
                     /           \            \
                    /             \            \
                   /\              \            \
                  (*)               (+)           (8) 
                 /   \            /      \
               (+)   (-)         /       (-)
              /  \   /  \       /       /   \ 
            (5) (2) (2) (1)   (+)     (-)   (1) 
                             /   \    / \     
                            (2) (9)  (7) (2)  
                  
                  

"""

In [ ]:
#-------------R-8-14----------------
"""

Operation
Running Time
len, is empty  O(1)   -- already stored hence O(1)
root, parent, is_root, is_leaf  O(1)   heighr level stored hence O(1)
children( p)  O(cp +1)  -- traversing to p is c(p) +1
depth(p) O(dp +1)  traversing to p depth o(p) + O(1)=O(p+1)
height  O(n) - need to traverse through root all the elements in the tree O(n)

"""

In [268]:
#-------------R-8-20----------------
class order(Node):
    def pre_order(self,root):
        result = []
        if root is not None:
            result.append(root._element)
            result = result + self.pre_order(root._left)
            result = result + self.pre_order(root._right)
        return result
    
    def inorder(self,root):
        result = []
        if root is not None:
            result = self.inorder(root._left)
            result.append(root._element)
            result = result + self.inorder(root._right)
        return result 
    
    def insert(self, e):
        """Compare the new value with the parent node"""
        if self._element:
            if e < self._element:
                if self._left is None:
                    self._left = Node(e)
                else:
                    self._left.insert(e)
            elif e > self._element:
                if self._right is None:
                    self._right = Node(e)
                else:
                    self._right.insert(e)
        else:
            self._element
        self._size += 1
         
    
    def printTree(self):
        if self._left is not None:
            self._left.PrintTree()
            print(chr(self._element) + ' ')
        if self._right is not None:
            self._right.PrintTree() 
           
     

root = order(ord('X'))
root.insert(ord('E'))
root.insert(ord('A'))
root.insert(ord('M'))
root.insert(ord('F'))
root.insert(ord('U'))
root.insert(ord('N'))
root.printTree()
print(root.pre_order(root))

65
69
70
77
78
85
X 
[88, 69, 65, 77, 70, 85, 78]
